In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.ensemble import VotingClassifier,VotingRegressor,StackingRegressor,StackingClassifier,RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression
from mlxtend.classifier import StackingClassifier as stc
from mlxtend.regressor import StackingRegressor as strg
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from joblib import dump

In [ ]:
cat_columns = ['Gauging Station_Badalgama','Gauging Station_Baddegama',
               'Gauging Station_Calidonia','Gauging Station_Deduru Oya',
               'Gauging Station_Deraniyagala','Gauging Station_Dunamale',
               'Gauging Station_Ellagawa','Gauging Station_Galgamuwa',
               'Gauging Station_Giriulla','Gauging Station_Glencourse',
               'Gauging Station_Hanwella','Gauging Station_Holombuwa',
               'Gauging Station_Horowpathana','Gauging Station_Kalawellawa',
               'Gauging Station_Katharagama','Gauging Station_Kitulgala',
               'Gauging Station_Kuda Oya','Gauging Station_Magura',
               'Gauging Station_Manampitiya(HMIS)','Gauging Station_Manampitiya(Old)',
               'Gauging Station_Moragaswewa','Gauging Station_Moraketiya',
               "Gauging Station_N' Street",'Gauging Station_Nakkala',
               'Gauging Station_Nawalapitiya','Gauging Station_Norwood',
               'Gauging Station_Padiyathalawa','Gauging Station_Panadugama',
               'Gauging Station_Peradeniya','Gauging Station_Pitabeddara',
               'Gauging Station_Putupaula','Gauging Station_Rathnapura',
               'Gauging Station_Ridi bendi Ella (Dam)','Gauging Station_Siyambalanduwa',
               'Gauging Station_Thaldena','Gauging Station_Thalgahagoda',
               'Gauging Station_Thanamalwila','Gauging Station_Thanthirimale',
               'Gauging Station_Thawalama','Gauging Station_Urawa',
               'Gauging Station_Wellawaya','Gauging Station_Weraganthota',
               'Gauging Station_Yaka Wewa']


flood_levels = {
    "N' Street": {"alert": 4, "minor_flood": 5, "major_flood": 7},
    "Hanwella": {"alert": 7, "minor_flood": 8, "major_flood": 10},
    "Glencourse": {"alert": 15, "minor_flood": 16.5, "major_flood": 19},
    "Kitulgala": {"alert": 3, "minor_flood": 4, "major_flood": 6},
    "Holombuwa": {"alert": 3, "minor_flood": 3.4, "major_flood": 5},
    "Deraniyagala": {"alert": 4.8, "minor_flood": 5.8, "major_flood": 6.4},
    "Norwood": {"alert": 1.5, "minor_flood": 3, "major_flood": 4.5},
    "Putupaula": {"alert": 3, "minor_flood": 4, "major_flood": 5},
    "Ellagawa": {"alert": 10, "minor_flood": 10.7, "major_flood": 12.2},
    "Rathnapura": {"alert": 5.2, "minor_flood": 7.5, "major_flood": 9.5},
    "Magura": {"alert": 4, "minor_flood": 6, "major_flood": 7.5},
    "Kalawellawa": {"alert": 5, "minor_flood": 6.5, "major_flood": 8},
    "Thawalama": {"alert": 4, "minor_flood": 6, "major_flood": 7.5},
    "Baddegama": {"alert": 3.5, "minor_flood": 4, "major_flood": 5},
    "Thalgahagoda": {"alert": 1.4, "minor_flood": 1.7, "major_flood": 2.8},
    "Panadugama": {"alert": 5, "minor_flood": 6, "major_flood": 7.5},
    "Pitabeddara": {"alert": 4, "minor_flood": 5, "major_flood": 6.5},
    "Urawa": {"alert": 2.5, "minor_flood": 4, "major_flood": 6},
    "Moraketiya": {"alert": 1.8, "minor_flood": 3.5, "major_flood": 5},
    "Thanamalwila": {"alert": 4, "minor_flood": 5, "major_flood": 5.5},
    "Wellawaya": {"alert": 4.4, "minor_flood": 5.4, "major_flood": 5.9},
    "Kuda Oya": {"alert": 6.9, "minor_flood": 8.4, "major_flood": 8.8},
    "Katharagama": {"alert": 4, "minor_flood": 4.6, "major_flood": 6.5},
    "Nakkala": {"alert": 5, "minor_flood": 6, "major_flood": 7.5},
    "Siyambalanduwa": {"alert": 4.5, "minor_flood": 6, "major_flood": 7},
    "Padiyathalawa": {"alert": 4, "minor_flood": 4.5, "major_flood": 6},
    "Manampitiya (Old)": {"alert": 3.5, "minor_flood": 5, "major_flood": 6},
    "Manampitiya (HMIS)": {"alert": 4.65, "minor_flood": 6.15, "major_flood": 7.15},
    "Weraganthota": {"alert": 5, "minor_flood": 6, "major_flood": 8},
    "Peradeniya": {"alert": 5, "minor_flood": 7, "major_flood": 9},
    "Nawalapitiya": {"alert": 3.5, "minor_flood": 5, "major_flood": 6},
    "Thaldena": {"alert": 3, "minor_flood": 4, "major_flood": 5},
    "Calidonia": {"alert": 3, "minor_flood": 3.5, "major_flood": 4.5},
    "Horowpathana": {"alert": 6, "minor_flood": 7.5, "major_flood": 10.5},
    "Yaka Wewa": {"alert": 4, "minor_flood": 5, "major_flood": 6},
    "Thantirimale": {"alert": 5, "minor_flood": 6.8, "major_flood": 7.8},
    "Galgamuwa": {"alert": 4.84, "minor_flood": 5.94, "major_flood": 8},
    "Moragaswewa":{"alert":4.5,"minor_flood":6.8,"major_flood":8.2 },
    "Ridi bendi Ella (Dam)": {"alert": 3.3, "minor_flood": 4.2, "major_flood": 4.7},
    "Badalgama": {"alert": 5, "minor_flood": 6.2, "major_flood": 9.6},
    "Giriulla": {"alert": 5.5, "minor_flood": 6.5, "major_flood": 7.5},
    "Dunamale": {"alert": 3.3, "minor_flood": 4.4, "major_flood": 5.5}
}
print(len(cat_columns))
print(len(flood_levels))

In [ ]:
def determine_flood_status(region, water_level):
    thresholds = flood_levels.get(region)
    

    if water_level < thresholds["alert"]:
        return "Normal"
    elif water_level < thresholds["minor_flood"]:
        return "Alert"
    elif water_level < thresholds["major_flood"]:
        return "Minor Flood"
    elif water_level >= thresholds["major_flood"]:
        return "Major Flood"
    return "Unknown"

In [78]:
import pandas as pd

df = pd.read_csv(r"C:\Users\User\Desktop\Flood Prediction Model\Flood Prediction Model\dataset(Rainy).csv")
df['24hr Rainfall'] = pd.to_numeric(df['24hr Rainfall'], errors='coerce')

highest_rainfall_df = df.loc[df.groupby(['Gauging Station', 'Date'])['24hr Rainfall'].idxmax()]

print(highest_rainfall_df)


                             River Gauging Station      Date        Time  \
6131                      Maha Oya       Badalgama  1-Aug-22  9:30:00 AM   
3808                      Maha Oya       Badalgama  1-Dec-21  9:30:00 AM   
20912                     Maha Oya       Badalgama  1-Jan-24  9:30:00 AM   
5688                      Maha Oya       Badalgama  1-Jun-22  9:30:00 AM   
25861                     Maha Oya       Badalgama  1-Jun-24  6:30:00 PM   
...                            ...             ...       ...         ...   
8589   Maa Oya (RB 69)  Mukunu Oya       Yaka Wewa  9-Nov-22  9:30:00 AM   
17313  Maa Oya (RB 69)  Mukunu Oya       Yaka Wewa  9-Nov-23  9:30:00 AM   
14175  Maa Oya (RB 69)  Mukunu Oya       Yaka Wewa  9-Oct-23  9:30:00 AM   
7339   Maa Oya (RB 69)  Mukunu Oya       Yaka Wewa  9-Sep-22  9:30:00 AM   
11533  Maa Oya (RB 69)  Mukunu Oya       Yaka Wewa  9-Sep-23  9:30:00 AM   

       Water level 1 our ago  Water level 1hr after  Risk  24hr Rainfall  
6131        

In [51]:
model=RandomForestRegressor(n_estimators=300,max_depth=30,min_samples_split=10)

In [52]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False), ['Gauging Station'])
    ],
    remainder='passthrough'
)

In [53]:
transformed_data = preprocessor.fit_transform(highest_rainfall_df)
cat_columns = preprocessor.named_transformers_['cat'].get_feature_names_out(['Gauging Station'])

new_columns = list(cat_columns) + ['Gauging Station','Date', 'Time', 'Water level 1 our ago','Water level 1hr after','Risk','24hr Rainfall']

transformed_df = pd.DataFrame(transformed_data, columns=new_columns)

X = transformed_df[cat_columns.tolist() + ['24hr Rainfall']]
y=transformed_df[['Water level 1 our ago','Water level 1hr after']]

In [54]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model.fit(X_train,y_train)

RandomForestRegressor(max_depth=30, min_samples_split=10, n_estimators=300)

In [55]:
y_pred=model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)#0.6895,R=0.7881,MSE=1.1203,RMSE=1.058
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Model Performance:")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R²): {r2:.4f}")

Model Performance:
Mean Absolute Error (MAE): 0.5969
Mean Squared Error (MSE): 0.8289
Root Mean Squared Error (RMSE): 0.9104
R-squared (R²): 0.8175


In [ ]:
# import joblib
# joblib.dump(model, 'model1.joblib')

In [56]:
def predict_water_levels(gauging_station, rainfall):

    input_data = {col: 0 for col in cat_columns}
    gauging_station_column = f'Gauging Station_{gauging_station}'
    print(gauging_station_column)
    
    if gauging_station_column in input_data:
        input_data[gauging_station_column] = 1
    else:
        raise ValueError(f"Invalid Gauging Station: {gauging_station}")

    input_data['24hr Rainfall'] = rainfall
    input_df = pd.DataFrame([input_data])
    
    return model.predict(input_df)

In [57]:
region="Kalawellawa"
predictions = predict_water_levels(region,149.8)
print("Predicted Water Level 1 hour ago:", predictions[0][0])
print("Predicted Water Level 1hr after:", predictions[0][1])
print("Risk Status:",determine_flood_status(region,predictions[0][1]))

Gauging Station_Kalawellawa
Predicted Water Level 1 hour ago: 3.8523464976717157
Predicted Water Level 1hr after: 3.8552549260625026
Risk Status: Normal
